In [1]:
from fancyimpute import KNN,NuclearNormMinimization, SoftImpute, BiScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
#from fancyimpute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
%matplotlib inline

Using TensorFlow backend.


In [2]:
#使用カラムを絞る
usecols=['Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Distance', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']

In [3]:
df=pd.read_csv("melbourne-housing-market/Melbourne_housing_FULL.csv",usecols=usecols)
df=df[df["Price"]==df["Price"]] #目的変数がないものを除外

In [4]:
df.isnull().sum()

Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Distance             1
Bedroom2          6441
Bathroom          6447
Car               6824
Landsize          9265
BuildingArea     16591
YearBuilt        15163
Propertycount        3
dtype: int64

In [5]:
#カテゴリ値を数値変換
le = preprocessing.LabelEncoder()
df[[ 'Type', 'Method', 'SellerG']]=df[[ 'Type', 'Method', 'SellerG']].apply(le.fit_transform)
df=df.dropna(subset=["Price"])

In [6]:
df_train=df.drop("Price",axis=1)
df_label=df["Price"].values

In [7]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
import copy

def run_model(df_train,df_label):
    #params={}
    params={
        'objective': 'regression',
        'random_state' : 1,
        "metric": "rmse",
        }

    kfold = 5
    score=0
    #sss = StratifiedShuffleSplit(n_splits=kfold, test_size=0.2, random_state=94)
    #sss = StratifiedKFold(n_splits=5, random_state=94)
    kf = KFold(n_splits=kfold,shuffle=True,random_state=94)
    
    importance=0
    for i, (train_index, test_index) in enumerate(kf.split(df_train, df_label)):
        print('[Fold %d/%d]' % (i + 1, kfold))

        X_train, X_valid = df_train.iloc[train_index,:], df_train.iloc[test_index,:]
        y_train, y_valid = df_label[train_index], df_label[test_index]
        dtrain = lgb.Dataset(X_train, label=y_train)
        dvalid = lgb.Dataset(X_valid, label=y_valid)

        bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid],early_stopping_rounds=50,verbose_eval=100)
        importance+= pd.DataFrame(bst.feature_importance(), index=df_train.columns, columns=['importance'])
        score+=bst.best_score["valid_1"]['rmse']
    return importance,score/5

In [141]:
imp,score=run_model(df_train,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 263065	valid_1's rmse: 361421
[200]	training's rmse: 234052	valid_1's rmse: 352385
[300]	training's rmse: 216683	valid_1's rmse: 349947
[400]	training's rmse: 204275	valid_1's rmse: 349206
Early stopping, best iteration is:
[380]	training's rmse: 206508	valid_1's rmse: 348965
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 271639	valid_1's rmse: 320021
[200]	training's rmse: 242773	valid_1's rmse: 311486
[300]	training's rmse: 225266	valid_1's rmse: 308975
[400]	training's rmse: 212653	valid_1's rmse: 308360
Early stopping, best iteration is:
[353]	training's rmse: 218298	valid_1's rmse: 308194
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 275512	valid_1's rmse: 301153
[200]	training's rmse: 245402	valid_1's rmse: 292408
[300]	training's rmse: 227787	valid_1's rmse: 288409
[400]	training's rmse:

In [144]:
#欠損値補完(baysian ridge)
imp = IterativeImputer(max_iter=50, random_state=5)
df_train_f1=pd.DataFrame(imp.fit_transform(df_train))
df_train_f1.columns=df_train.columns
imp,score=run_model(df_train_f1,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 262690	valid_1's rmse: 365918
[200]	training's rmse: 231042	valid_1's rmse: 360730
[300]	training's rmse: 210424	valid_1's rmse: 358058
Early stopping, best iteration is:
[337]	training's rmse: 204343	valid_1's rmse: 357014
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 271661	valid_1's rmse: 330273
[200]	training's rmse: 240099	valid_1's rmse: 322404
[300]	training's rmse: 220261	valid_1's rmse: 319745
[400]	training's rmse: 205034	valid_1's rmse: 319453
Early stopping, best iteration is:
[363]	training's rmse: 210627	valid_1's rmse: 319082
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 277991	valid_1's rmse: 304364
[200]	training's rmse: 243125	valid_1's rmse: 295271
[300]	training's rmse: 222255	valid_1's rmse: 292180
Early stopping, best iteration is:
[272]	training's rmse: 227058	valid_1's 

In [145]:
#欠損値補完(ExtraTreesRegressor)
from sklearn.ensemble import ExtraTreesRegressor
imp = IterativeImputer(estimator=ExtraTreesRegressor(),max_iter=10, random_state=0)
#imp = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=15),max_iter=10, random_state=0)
df_train_f2=pd.DataFrame(imp.fit_transform(df_train))
df_train_f2.columns=df_train.columns
imp,score=run_model(df_train_f2,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 263039	valid_1's rmse: 369032
[200]	training's rmse: 228840	valid_1's rmse: 359115
[300]	training's rmse: 208692	valid_1's rmse: 356194
[400]	training's rmse: 193218	valid_1's rmse: 355391
[500]	training's rmse: 180858	valid_1's rmse: 354484
[600]	training's rmse: 170482	valid_1's rmse: 354039
Early stopping, best iteration is:
[591]	training's rmse: 171497	valid_1's rmse: 353921
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 273241	valid_1's rmse: 336999
[200]	training's rmse: 238961	valid_1's rmse: 325400
[300]	training's rmse: 217199	valid_1's rmse: 321088
[400]	training's rmse: 202296	valid_1's rmse: 320012
[500]	training's rmse: 189526	valid_1's rmse: 319824
Early stopping, best iteration is:
[485]	training's rmse: 191223	valid_1's rmse: 319734
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse:

In [146]:
#欠損値補完(KNeighborsRegressor)
from sklearn.neighbors import KNeighborsRegressor
imp = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=15),max_iter=10, random_state=0)
df_train_f3=pd.DataFrame(imp.fit_transform(df_train))
df_train_f3.columns=df_train.columns
imp,score=run_model(df_train_f3,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 263979	valid_1's rmse: 366423
[200]	training's rmse: 233338	valid_1's rmse: 359232
[300]	training's rmse: 213555	valid_1's rmse: 357730
[400]	training's rmse: 198665	valid_1's rmse: 356891
Early stopping, best iteration is:
[428]	training's rmse: 195279	valid_1's rmse: 356451
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 275127	valid_1's rmse: 333599
[200]	training's rmse: 244002	valid_1's rmse: 324737
[300]	training's rmse: 224356	valid_1's rmse: 322026
[400]	training's rmse: 209479	valid_1's rmse: 320888
[500]	training's rmse: 197568	valid_1's rmse: 320396
[600]	training's rmse: 186955	valid_1's rmse: 320142
[700]	training's rmse: 178698	valid_1's rmse: 319649
Early stopping, best iteration is:
[675]	training's rmse: 180625	valid_1's rmse: 319341
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse:

In [148]:
#fancyimpute(MF)
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,MatrixFactorization
df_train_f4 = pd.DataFrame(MatrixFactorization(verbose=False).fit_transform(df_train))
df_train_f4.columns=df_train.columns
imp,score=run_model(df_train_f4,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 268683	valid_1's rmse: 372455
[200]	training's rmse: 235216	valid_1's rmse: 364434
[300]	training's rmse: 212294	valid_1's rmse: 362866
[400]	training's rmse: 195695	valid_1's rmse: 361702
Early stopping, best iteration is:
[401]	training's rmse: 195614	valid_1's rmse: 361673
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 280192	valid_1's rmse: 337948
[200]	training's rmse: 246761	valid_1's rmse: 329144
[300]	training's rmse: 225308	valid_1's rmse: 327315
[400]	training's rmse: 209132	valid_1's rmse: 325621
[500]	training's rmse: 195514	valid_1's rmse: 324928
Early stopping, best iteration is:
[505]	training's rmse: 195014	valid_1's rmse: 324924
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 283447	valid_1's rmse: 306800
[200]	training's rmse: 249608	valid_1's rmse: 298163
[300]	training's rmse:

In [13]:
#fancyimpute(BiScaler)
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,MatrixFactorization
df_train_f5 = pd.DataFrame(BiScaler(verbose=False).fit_transform(np.array(df_train)))
df_train_f5.columns=df_train.columns
imp,score=run_model(df_train_f5,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 320990	valid_1's rmse: 434285
[200]	training's rmse: 288714	valid_1's rmse: 430946
Early stopping, best iteration is:
[229]	training's rmse: 280741	valid_1's rmse: 429541
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 330353	valid_1's rmse: 396107
[200]	training's rmse: 298296	valid_1's rmse: 391989
[300]	training's rmse: 273526	valid_1's rmse: 389982
Early stopping, best iteration is:
[282]	training's rmse: 277954	valid_1's rmse: 389522
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 334701	valid_1's rmse: 378164
[200]	training's rmse: 303039	valid_1's rmse: 374696
[300]	training's rmse: 278711	valid_1's rmse: 374393
Early stopping, best iteration is:
[279]	training's rmse: 283504	valid_1's rmse: 374185
[Fold 4/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rms

In [14]:
#fancyimpute(SoftImpute)
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, BiScaler,MatrixFactorization
df_train_f6 = pd.DataFrame(SoftImpute(verbose=False).fit_transform(df_train))
df_train_f6.columns=df_train.columns
imp,score=run_model(df_train_f6,df_label)
print(score)

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 266793	valid_1's rmse: 363229
[200]	training's rmse: 237589	valid_1's rmse: 354736
[300]	training's rmse: 219186	valid_1's rmse: 351821
[400]	training's rmse: 205788	valid_1's rmse: 351039
[500]	training's rmse: 194860	valid_1's rmse: 350464
Early stopping, best iteration is:
[518]	training's rmse: 192966	valid_1's rmse: 350300
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 275779	valid_1's rmse: 328882
[200]	training's rmse: 245698	valid_1's rmse: 319593
[300]	training's rmse: 226554	valid_1's rmse: 315814
[400]	training's rmse: 213474	valid_1's rmse: 314462
Early stopping, best iteration is:
[428]	training's rmse: 210404	valid_1's rmse: 313974
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 280563	valid_1's rmse: 303490
[200]	training's rmse: 249417	valid_1's rmse: 294938
[300]	training's rmse: